In [20]:
from IPython.display import display
import IPython
import pandas as pd
from pandas import Series
import numpy as np
from random import randrange
from matplotlib import pyplot
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import math
from typing import Any, List, Dict, AnyStr, Optional
from pathlib import Path
from glob import glob
from utils.dataset import MarketData, DataPreprocessor, Plotting
from utils.windowgenerator import WindowGenerator, compile_and_fit
import tensorflow as tf
import talib
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
eua_options_cols = ['Date', 'Aggregate Put Open Interest  (R1)', 'Aggregate Call Open Interest  (R1)', 
                    'Aggregate Open Interest  (L1)', 'OPTION OI%', 'PUT/CALL OI']
eua_options = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='EUA option-G363')
eua_options['Date'] = pd.to_datetime(pd.to_datetime(eua_options['Date']).dt.date)
eua_options = eua_options[eua_options_cols][eua_options['Date'].dt.year >= 2018].dropna()

In [69]:
display(eua_options.head())

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI
1,2024-10-31,206996.0,321757.0,763691.0,0.409111,1.554412
2,2024-10-30,204588.0,322482.0,759379.0,0.409709,1.576251
3,2024-10-29,204538.0,320432.0,760706.0,0.408322,1.566614
4,2024-10-28,204623.0,321162.0,785605.0,0.400937,1.569530
5,2024-10-25,204723.0,321187.0,786404.0,0.400750,1.568886


In [70]:
display(eua_options.tail())

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI
1086,2020-08-18,5150.0,28350.0,954401.0,0.033910,5.504854
1087,2020-08-17,5150.0,28350.0,950182.0,0.034056,5.504854
1088,2020-08-14,5150.0,28350.0,946283.0,0.034191,5.504854
1089,2020-08-13,5150.0,28350.0,945581.0,0.034216,5.504854
1090,2020-08-12,5150.0,28350.0,944604.0,0.034250,5.504854


In [151]:
pachis_delta = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='25Delta')
dec_cols = ["Date", "Hist Vol", "50D-Hist Vol", "50D", "25D Spread", "butterfly"]
pachis_delta_dec = pachis_delta[dec_cols]
pachis_delta_dec.columns = ["Date", "Hist Vol - 1Y", 
                        "iVol/Hist Vol Spread - Dec", 
                        "50 Delta iVol - Dec", 
                        "25Δ Risk Reversal (Call - Put) - Dec", 
                        "Butterfly - Dec"]
pachis_delta_dec['Date'] = pd.to_datetime(pachis_delta_dec['Date'])
pachis_delta_dec = pachis_delta_dec[pachis_delta_dec['Date'].dt.year > 2018] 
for col in ['Hist Vol - 1Y', 'iVol/Hist Vol Spread - Dec', '50 Delta iVol - Dec']:
    pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)
pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc('iVol/Hist Vol Spread - Dec')] = pachis_delta_dec['iVol/Hist Vol Spread - Dec'].astype(float)
pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc('50 Delta iVol - Dec')] = pachis_delta_dec['50 Delta iVol - Dec'].astype(float)


/var/folders/zf/km2fq0xs06g8b5nj2t36d5fw0000gn/T/ipykernel_75578/307972202.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pachis_delta_dec['Date'] = pd.to_datetime(pachis_delta_dec['Date'])
/var/folders/zf/km2fq0xs06g8b5nj2t36d5fw0000gn/T/ipykernel_75578/307972202.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)
/var/folders/zf/km2fq0xs06g8b5nj2t36d5fw0000gn/T/ipykernel_75578/307972202.py:

In [157]:
prompt_cols = ["Date.1", "50D.1", "25D Spread.1", "butterfly.1"]
pachis_delta_prompt = pachis_delta[prompt_cols]
pachis_delta_prompt.columns = ["Date", "50 Delta iVol - Prompt",
                               "25Δ Risk Reversal (Call - Put) - Prompt",
                               "Butterfly - Prompt"]
pachis_delta_prompt['Date'] = pd.to_datetime(pachis_delta_prompt['Date'])
pachis_delta_prompt = pachis_delta_prompt[pachis_delta_prompt['Date'].dt.year > 2018]
# First convert any string values to numeric, handling spaces
pachis_delta_prompt['50 Delta iVol - Prompt'] = pd.to_numeric(pachis_delta_prompt['50 Delta iVol - Prompt'].replace(' ', np.nan), errors='coerce')

/var/folders/zf/km2fq0xs06g8b5nj2t36d5fw0000gn/T/ipykernel_75578/2935795338.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pachis_delta_prompt['Date'] = pd.to_datetime(pachis_delta_prompt['Date'])


In [161]:
option_ts = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='Option Time series')

In [209]:
# Create prompt dataframe
cols = ['Date', 'Call_OI-Prompt', 'Put_OI-Prompt', 'Dec_OI', 'Call/Put', 'Option%']
option_ts_prompt = option_ts[cols].copy()
option_ts_prompt.columns = ['Date', 'Call_OI-Prompt', 'Put_OI-Prompt', 'Dec_OI', 'Call/Put-Prompt', 'Option%-Prompt']

# Create december dataframe
cols = ['Date', 'Call/Put.1', 'Option%.1']
option_ts_dec = option_ts[cols].copy()
option_ts_dec.columns = ['Date', 'Call/Put-Dec', 'Option%-Dec']

# Merge the two dataframes on Date
option_ts_combined = pd.merge(option_ts_prompt, option_ts_dec, on='Date', how='outer')
option_ts_combined = option_ts_combined.sort_values(by='Date', ascending=False)
option_ts_combined = option_ts_combined[:-74]

In [235]:
combined_df = eua_options.merge(pachis_delta_dec, on='Date', how='left')\
    .merge(pachis_delta_prompt, on='Date', how='left')\
    .merge(option_ts_combined, on='Date', how='left')

In [257]:
combined_df = combined_df[combined_df['Date'].dt.year>=2023][:-70]

In [259]:
combined_df

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI,Hist Vol - 1Y,iVol/Hist Vol Spread - Dec,50 Delta iVol - Dec,25Δ Risk Reversal (Call - Put) - Dec,...,50 Delta iVol - Prompt,25Δ Risk Reversal (Call - Put) - Prompt,Butterfly - Prompt,Call_OI-Prompt,Put_OI-Prompt,Dec_OI,Call/Put-Prompt,Option%-Prompt,Call/Put-Dec,Option%-Dec
0,2024-10-31,206996.0,321757.0,763691.0,0.409111,1.554412,28.706,8.366,37.072,-3.106,...,37.072,-3.106,1.1600,143541.0,111708.0,381074.0,1.284966,0.401131,1.284966,0.401131
1,2024-10-30,204588.0,322482.0,759379.0,0.409709,1.576251,28.694,8.398,37.092,-2.899,...,37.092,-2.899,0.9245,143741.0,111708.0,377423.0,1.286757,0.403635,1.286757,0.403635
2,2024-10-29,204538.0,320432.0,760706.0,0.408322,1.566614,28.205,8.773,36.978,-2.460,...,36.978,-2.460,1.0570,144571.0,111993.0,379961.0,1.290893,0.403070,1.290893,0.403070
3,2024-10-28,204623.0,321162.0,785605.0,0.400937,1.569530,28.121,8.829,36.95,-2.710,...,36.950,-2.710,1.1120,144571.0,112093.0,383677.0,1.289742,0.400824,1.289742,0.400824
4,2024-10-25,204723.0,321187.0,786404.0,0.400750,1.568886,28.34,8.534,36.874,-2.563,...,36.874,-2.563,1.0375,142771.0,111895.0,385602.0,1.275937,0.397749,1.275937,0.397749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,2023-04-20,104186.0,108732.0,503017.0,0.297399,1.043634,35.089,8.63,43.719,-3.627,...,41.104,-3.992,1.4260,18136.0,42833.0,259099.0,0.423412,0.190488,0.871061,0.297488
425,2023-04-19,102023.0,108612.0,502064.0,0.295546,1.064583,36.672,6.921,43.593,-3.359,...,40.937,-2.339,1.0805,16913.0,43758.0,262545.0,0.386512,0.187710,0.871061,0.294734
426,2023-04-18,102948.0,107389.0,502993.0,0.294866,1.043138,37.102,6.516,43.618,-2.995,...,41.498,-2.498,1.1590,16286.0,43460.0,264073.0,0.374735,0.184504,0.857175,0.291634
427,2023-04-17,102550.0,104912.0,502072.0,0.292392,1.023033,37.981,5.7,43.681,-3.051,...,42.012,-3.113,1.0855,16286.0,43360.0,265414.0,0.375600,0.183492,0.857418,0.290968


In [1]:
from IPython.display import display
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path
from pymongo import MongoClient
from datetime import datetime

from utils.dataset import MarketData, DataPreprocessor
from utils.data_processing import prepare_data

ModuleNotFoundError: No module named 'utils'

In [263]:
def load_and_preprocess_data():
    cot_df, auction_df, options_df, ta_df, fundamentals_df = MarketData.latest(Path('../data'))
    cot_df = cot_df.set_index('Date').resample('W', origin='end').mean().reset_index()
    auction_df = auction_df.set_index('Date').resample('D').mean().reset_index()

    auction_df = auction_df[7:]
    # auction_df.loc[:, 'Premium/discount-settle'] = auction_df['Premium/discount-settle'].ffill()
    auc_cols = ['Auc Price', 'Median Price', 'Cover Ratio', 'Spot Value', 
                'Auction Spot Diff', 'Median Spot Diff', 'Premium/discount-settle']
    auction_df.loc[:, auc_cols] = auction_df[auc_cols].ffill()

    merged_df = DataPreprocessor.engineer_auction_features(auction_df)
    return merged_df, options_df

In [266]:
merged_df, options_df = load_and_preprocess_data()

/Users/zeerakwyne/Documents/freelance/carbon-market-forecasting/notebooks/../utils/dataset.py:106: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)
/Users/zeerakwyne/Documents/freelance/carbon-market-forecasting/notebooks/../utils/dataset.py:106: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)


In [265]:

options_df

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI,Hist Vol - 1Y,iVol/Hist Vol Spread - Dec,50 Delta iVol - Dec,25Δ Risk Reversal (Call - Put) - Dec,...,50 Delta iVol - Prompt,25Δ Risk Reversal (Call - Put) - Prompt,Butterfly - Prompt,Call_OI-Prompt,Put_OI-Prompt,Dec_OI,Call/Put-Prompt,Option%-Prompt,Call/Put-Dec,Option%-Dec
0,2024-10-31,206996.0,321757.0,763691.0,0.409111,1.554412,28.706,8.366,37.072,-3.106,...,37.072,-3.106,1.1600,143541.0,111708.0,381074.0,1.284966,0.401131,1.284966,0.401131
1,2024-10-30,204588.0,322482.0,759379.0,0.409709,1.576251,28.694,8.398,37.092,-2.899,...,37.092,-2.899,0.9245,143741.0,111708.0,377423.0,1.286757,0.403635,1.286757,0.403635
2,2024-10-29,204538.0,320432.0,760706.0,0.408322,1.566614,28.205,8.773,36.978,-2.460,...,36.978,-2.460,1.0570,144571.0,111993.0,379961.0,1.290893,0.403070,1.290893,0.403070
3,2024-10-28,204623.0,321162.0,785605.0,0.400937,1.569530,28.121,8.829,36.95,-2.710,...,36.950,-2.710,1.1120,144571.0,112093.0,383677.0,1.289742,0.400824,1.289742,0.400824
4,2024-10-25,204723.0,321187.0,786404.0,0.400750,1.568886,28.34,8.534,36.874,-2.563,...,36.874,-2.563,1.0375,142771.0,111895.0,385602.0,1.275937,0.397749,1.275937,0.397749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,2023-04-20,104186.0,108732.0,503017.0,0.297399,1.043634,35.089,8.63,43.719,-3.627,...,41.104,-3.992,1.4260,18136.0,42833.0,259099.0,0.423412,0.190488,0.871061,0.297488
425,2023-04-19,102023.0,108612.0,502064.0,0.295546,1.064583,36.672,6.921,43.593,-3.359,...,40.937,-2.339,1.0805,16913.0,43758.0,262545.0,0.386512,0.187710,0.871061,0.294734
426,2023-04-18,102948.0,107389.0,502993.0,0.294866,1.043138,37.102,6.516,43.618,-2.995,...,41.498,-2.498,1.1590,16286.0,43460.0,264073.0,0.374735,0.184504,0.857175,0.291634
427,2023-04-17,102550.0,104912.0,502072.0,0.292392,1.023033,37.981,5.7,43.681,-3.051,...,42.012,-3.113,1.0855,16286.0,43360.0,265414.0,0.375600,0.183492,0.857418,0.290968


In [27]:
from IPython.display import display
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pathlib import Path
from pymongo import MongoClient
from datetime import datetime
import sys
sys.path.append("../")
from utils.dataset import MarketData, DataPreprocessor
from utils.data_processing import prepare_data, reverse_normalize
from utils.plotting import (display_performance_metrics, display_trade_log, plot_equity_curve, 
                            plot_model_results_with_trades, plot_recent_predictions, plot_ensemble_statistics, plot_ensemble_predictions_realtime)
from utils.model_utils import create_model, train_model, generate_model_predictions, train_ensemble_models
from utils.mongodb_utils import get_stored_predictions, setup_mongodb_connection, save_recent_predictions
from utils.backtesting import backtest_model_with_metrics

In [61]:
cot_df, auction_df, options_df, ta_df, fundamentals_df = MarketData.latest(Path('../data'))
cot_df = cot_df.set_index('Date').resample('W', origin='end').mean().reset_index()
auction_df = auction_df.set_index('Date').resample('D').mean().reset_index()

auction_df = auction_df[7:]
# auction_df.loc[:, 'Premium/discount-settle'] = auction_df['Premium/discount-settle'].ffill()
auc_cols = ['Auc Price', 'Median Price', 'Cover Ratio', 'Spot Value', 
            'Auction Spot Diff', 'Median Spot Diff', 'Premium/discount-settle']
auction_df.loc[:, auc_cols] = auction_df[auc_cols].ffill()

merged_df = DataPreprocessor.engineer_auction_features(auction_df)

/Users/zeerakwyne/Documents/freelance/carbon-market-forecasting/notebooks/../utils/dataset.py:106: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)
/Users/zeerakwyne/Documents/freelance/carbon-market-forecasting/notebooks/../utils/dataset.py:106: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pachis_delta_dec.iloc[:, pachis_delta_dec.columns.get_loc(col)] = pachis_delta_dec[col].replace(' ', np.nan).astype(float)


In [67]:
options_df.shape

(429, 21)

In [80]:
dup_dates = options_df['Date'].value_counts()[:10].index

In [81]:
dup_dates

DatetimeIndex(['2024-10-04', '2024-09-05', '2024-09-04', '2024-08-07',
               '2024-08-05', '2024-08-02', '2024-08-01', '2024-10-07',
               '2024-09-02', '2024-10-01'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [82]:
options_df[options_df['Date'].isin(dup_dates)]

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI,Hist Vol - 1Y,iVol/Hist Vol Spread - Dec,50 Delta iVol - Dec,25Δ Risk Reversal (Call - Put) - Dec,...,50 Delta iVol - Prompt,25Δ Risk Reversal (Call - Put) - Prompt,Butterfly - Prompt,Call_OI-Prompt,Put_OI-Prompt,Dec_OI,Call/Put-Prompt,Option%-Prompt,Call/Put-Dec,Option%-Dec
18,2024-10-07,195213.0,292641.0,772488.0,0.387081,1.499086,26.181,10.132,36.313,-3.688,...,36.313,-3.688,1.0280,126399.0,110778.0,393040.0,1.141012,0.376342,1.141012,0.376342
19,2024-10-07,195213.0,292641.0,772488.0,0.387081,1.499086,26.181,10.132,36.313,-3.688,...,NaN,-1.354,0.6910,126399.0,110778.0,393040.0,1.141012,0.376342,1.141012,0.376342
20,2024-10-07,195213.0,292641.0,772488.0,0.387081,1.499086,34.005,0.363,34.368,-2.625,...,36.313,-3.688,1.0280,126399.0,110778.0,393040.0,1.141012,0.376342,1.141012,0.376342
21,2024-10-07,195213.0,292641.0,772488.0,0.387081,1.499086,34.005,0.363,34.368,-2.625,...,NaN,-1.354,0.6910,126399.0,110778.0,393040.0,1.141012,0.376342,1.141012,0.376342
22,2024-10-04,195513.0,292041.0,793944.0,0.380456,1.493717,26.216,9.696,35.912,-4.089,...,35.912,-4.089,0.5945,126064.0,110853.0,414837.0,1.137218,0.363507,1.137218,0.363507
23,2024-10-04,195513.0,292041.0,793944.0,0.380456,1.493717,26.216,9.696,35.912,-4.089,...,41.312,-3.944,1.4790,126064.0,110853.0,414837.0,1.137218,0.363507,1.137218,0.363507
24,2024-10-04,195513.0,292041.0,793944.0,0.380456,1.493717,46.116,-6.614,39.502,-5.390,...,35.912,-4.089,0.5945,126064.0,110853.0,414837.0,1.137218,0.363507,1.137218,0.363507
25,2024-10-04,195513.0,292041.0,793944.0,0.380456,1.493717,46.116,-6.614,39.502,-5.390,...,41.312,-3.944,1.4790,126064.0,110853.0,414837.0,1.137218,0.363507,1.137218,0.363507
28,2024-10-01,189348.0,290131.0,773668.0,0.382620,1.532263,26.515,8.408,34.923,-3.956,...,34.923,-3.956,0.9020,125734.0,110448.0,405300.0,1.138400,0.368182,1.138400,0.368182
29,2024-10-01,189348.0,290131.0,773668.0,0.382620,1.532263,26.515,8.408,34.923,-3.956,...,40.926,-4.579,1.2425,125734.0,110448.0,405300.0,1.138400,0.368182,1.138400,0.368182


In [57]:
auc_df = merged_df[['Date', 'Auc Price']].copy()
options_df = options_df.merge(auc_df, how='inner')
options_df = options_df.bfill()

/var/folders/zf/km2fq0xs06g8b5nj2t36d5fw0000gn/T/ipykernel_26185/555126379.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  options_df = options_df.bfill()


In [47]:
options_df = options_df.set_index('Date')

In [52]:
options_df.shape

(1330, 22)

In [54]:
options_df['Date'].value_counts()

Date
2023-12-23    17
2024-01-01    17
2024-01-12    17
2024-01-11    17
2024-01-10    17
              ..
2024-03-21     1
2024-03-24     1
2024-03-23     1
2024-03-22     1
2023-04-16     1
Name: count, Length: 645, dtype: int64

In [17]:
def prepare_data_and_train_model(merged_df):
    train_df, test_df, val_df, preprocessor = prepare_data(merged_df)
    num_features = len(test_df.columns)
    OUT_STEPS = 7
    model = create_model(num_features, OUT_STEPS)
    history = train_model(model, train_df, val_df, test_df, preprocessor)
    predictions_df, recent_preds, trend = generate_model_predictions(model, test_df)
    
    return model, preprocessor, test_df, predictions_df, recent_preds, trend

In [18]:
train_df, test_df, val_df, preprocessor = prepare_data(options_df)

KeyError: 'Date'

In [6]:
num_features = len(test_df.columns)
OUT_STEPS = 7
model = create_model(num_features, OUT_STEPS)

In [7]:
history = train_model(model, train_df, val_df, test_df, preprocessor)

Epoch 1/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9657 - mean_absolute_error: 0.7771 - val_loss: 1.0725 - val_mean_absolute_error: 0.8215
Epoch 2/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6970 - mean_absolute_error: 0.6342 - val_loss: 0.7508 - val_mean_absolute_error: 0.6253
Epoch 3/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5419 - mean_absolute_error: 0.5302 - val_loss: 0.6203 - val_mean_absolute_error: 0.5515
Epoch 4/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4547 - mean_absolute_error: 0.4710 - val_loss: 0.5609 - val_mean_absolute_error: 0.5181
Epoch 5/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4162 - mean_absolute_error: 0.4479 - val_loss: 0.5276 - val_mean_absolute_error: 0.4884
Epoch 6/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4066 - mean_absolute_error: 0.4334 - val_loss: 0.4973 - val_mean_absolute_error: 0.4709
Epoch 7/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3783 - mean_absolute_error: 0.4125 - val_loss: 0.4553 - val_mean_a

In [8]:
predictions_df, recent_preds, trend = generate_model_predictions(model, test_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [9]:
trade_log_df, performance_metrics, balance_history_df = backtest_model_with_metrics(
                    model,
                    test_df,
                    7,
                    7,
    10000,
    4 / 100.0,
    2 / 100.0,
    position_size_fraction=100 / 100.0,
    risk_free_rate=1 / 100.0,
    preprocessor=preprocessor
)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
PREV PRICE: 62.3
PRED MEAN: 41.438480377197266
PREV PRICE: 63.3
PRED MEAN:

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [23]:
input_width = 7
out_steps= 7
initial_balance = 10000
take_profit = 4 / 100.0
stop_loss = 2 / 100.0
position_size_fraction =100 / 100.0
risk_free_rate=1 / 100.0

In [24]:
features = test_df.columns
num_features = len(features)
predictions = []

# Initialize balance and trade log
balance = initial_balance
balance_history = []
trade_log = []

# For performance metrics
returns = []
equity_curve = [initial_balance]

In [25]:
# Generate predictions in steps of 'out_steps'
for idx, i in enumerate(range(input_width, len(test_df) - out_steps + 1, out_steps)):
    try:
        inputs = test_df[i - input_width:i].values
        inputs_reshaped = inputs.reshape((1, input_width, num_features))
        preds = model.predict(inputs_reshaped)
        predictions.append(preds[0])
    except Exception as e:
        print(f"Prediction error at index {i}: {e}")
        break

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [28]:
# Create DataFrame for predictions
predictions = np.concatenate(predictions, axis=0)
pred_indices = test_df.index[input_width:input_width + len(predictions)]
predictions_df = pd.DataFrame(predictions, columns=features, index=pred_indices)
# Reverse normalize predictions and test data
predictions_df = reverse_normalize(predictions_df, preprocessor.train_mean['Auc Price'], preprocessor.train_std['Auc Price'])
test_df_denormalized = reverse_normalize(test_df.copy(), preprocessor.train_mean['Auc Price'], preprocessor.train_std['Auc Price'])


In [38]:
for idx, i in enumerate(range(input_width, len(test_df) - out_steps + 1, out_steps)):
    entry_index = predictions_df.index[idx * out_steps]
    entry_price = test_df_denormalized.loc[entry_index, 'Auc Price']
    prev_index = entry_index - pd.Timedelta(days=1)
    prev_price = test_df_denormalized.loc[prev_index, 'Auc Price'] if prev_index in test_df_denormalized.index else entry_price
    print(f"IDX: {idx} | i: {i}")
    print(f"Entry Index: {entry_index} | Entry Price: {entry_price} | ")
    print(f"Prev Index: {prev_index} | Prev Price: {prev_price}")
    print()
    if idx == 2: break

IDX: 0 | i: 7
Entry Index: 2024-10-22 00:00:00 | Entry Price: 61.22 | 
Prev Index: 2024-10-21 00:00:00 | Prev Price: 62.3

IDX: 1 | i: 14
Entry Index: 2024-10-11 00:00:00 | Entry Price: 63.71 | 
Prev Index: 2024-10-10 00:00:00 | Prev Price: 63.3

IDX: 2 | i: 21
Entry Index: 2024-10-07 00:00:00 | Entry Price: Date
2024-10-07    60.15
2024-10-07    60.15
2024-10-07    60.15
2024-10-07    60.15
Name: Auc Price, dtype: float64 | 
Prev Index: 2024-10-06 00:00:00 | Prev Price: Date
2024-10-07    60.15
2024-10-07    60.15
2024-10-07    60.15
2024-10-07    60.15
Name: Auc Price, dtype: float64

